In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.stats import zscore
import matplotlib.pyplot as plt
%matplotlib inline

# Load the dataset

In [2]:
df = pd.read_csv('ACMETelephoneABT.csv')

In [3]:
df.head()

,customer,age,occupation,regionType,marriageStatus,children,income,numHandsets,handsetAge,smartPhone,...,avgInCalls,peakOffPeakRatio,peakOffPeakRatioChangePct,avgDroppedCalls,lifeTime,lastMonthCustomerCareCalls,numRetentionCalls,numRetentionOffersAccepted,newFrequentNumbers,churn
0,1000004,26,crafts,town,yes,true,6,1,1812,false,...,0.00,0.362398,-6.330770,0.00,60,0.00,0,0,0,false
1,1000012,36,,,yes,true,9,4,544,true,...,1.67,5.586826,7.499560,0.00,53,0.00,0,0,0,false
2,1000034,74,professional,town,yes,false,7,2,138,true,...,0.33,0.500000,-5.940493,4.33,58,0.00,0,0,3,false
3,1000063,30,,suburban,no,false,6,3,122,true,...,4.67,8.629661,-0.066485,5.33,50,0.33,0,0,0,false
4,1000085,32,,town,yes,false,7,8,10,true,...,23.00,14.042685,-7.208502,14.67,56,0.00,0,0,0,false


In [4]:
df.shape

(10000, 33)

In [5]:
df['churn'].value_counts()

 false    5000
 true     5000
Name: churn, dtype: int64

# As we see, we have a perfectly balanced dataset, nothing further needs to be done to balance. 

# Customer adds to value to the dataframe, Churn is the target variable, hence dropping

In [6]:
#Dropping Customer column, as there is no value addition
df.drop(["customer","churn"],axis=1,inplace=True)

In [7]:
df.columns

Index([' age', ' occupation', ' regionType', 'marriageStatus', ' children',
       ' income', 'numHandsets', 'handsetAge', 'smartPhone',
       'currentHandsetPrice', ' creditRating', 'homeOwner', ' creditCard',
       'avgBill', 'avgMins', 'avgrecurringCharge', 'avgOverBundleMins',
       'avgRoamCalls', 'callMinutesChangePct', 'billAmountChangePct',
       'avgReceivedMins', 'avgOutCalls', 'avgInCalls', 'peakOffPeakRatio',
       'peakOffPeakRatioChangePct', 'avgDroppedCalls', 'lifeTime',
       'lastMonthCustomerCareCalls', 'numRetentionCalls',
       'numRetentionOffersAccepted', 'newFrequentNumbers'],
      dtype='object')

# As we see - there are column names with white spaces, stripping the white space below

In [8]:
df.columns=df.columns.str.strip()

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,10000.0,30.318400,22.158676,0.000000,0.000000,34.000000,48.000000,98.000000
income,10000.0,4.293600,3.139902,0.000000,0.000000,5.000000,7.000000,9.000000
numHandsets,10000.0,1.804500,1.345088,1.000000,1.000000,1.000000,2.000000,21.000000
handsetAge,10000.0,390.171700,257.076656,-5.000000,210.000000,339.000000,525.000000,1812.000000
currentHandsetPrice,10000.0,35.730696,57.072922,0.000000,0.000000,0.000000,59.990000,499.990000
avgBill,10000.0,58.927600,43.889815,0.000000,33.327500,49.205000,71.762500,584.230000
avgMins,10000.0,521.170645,540.435285,0.000000,150.627500,359.625000,709.187500,6336.250000
avgrecurringCharge,10000.0,46.236537,23.964960,0.000000,30.000000,44.990000,59.990000,337.980000
avgOverBundleMins,10000.0,42.392670,106.374374,0.000000,0.000000,3.000000,43.750000,4320.750000
avgRoamCalls,10000.0,1.186048,6.048811,0.000000,0.000000,0.000000,0.260000,177.990000


# From the above we can see, outliers are present 
Income shows 0 in the 25th percentile.
Age, handsetAge, AvgBill, lastMonthCustomerCareCalls - we observe outliers

Before treating the outliers, we see the data has blank fields, lets trim the strings and treat them.

In [10]:
trimStrings = lambda x: x.strip() if type(x) is str else x
df =df.applymap(trimStrings)

In [11]:
def getMissingColumns(df):
    'Method to get the dictionary with columns having missing values and count of missing values'
    na_columns = {}
    for col in df.columns:
        missed = df.shape[0] - df[col].dropna().shape[0]
        blanks = 0
        
        if(df[col].dtype == 'O'):
            blanks = len(df[df[col] == ''])
            
        if (missed > 0 or blanks>0):
            na_columns[col] = missed + blanks

    return na_columns

In [12]:
getMissingColumns(df)

{'occupation': 7400, 'regionType': 4776}

# Based on the null data - RegionType and Occupation is not a good data to be used for classification due to nulls, hence dropping

In [13]:
df.drop(["regionType","occupation"],axis=1,inplace=True)

In [14]:
df.head()

,age,marriageStatus,children,income,numHandsets,handsetAge,smartPhone,currentHandsetPrice,creditRating,homeOwner,...,avgOutCalls,avgInCalls,peakOffPeakRatio,peakOffPeakRatioChangePct,avgDroppedCalls,lifeTime,lastMonthCustomerCareCalls,numRetentionCalls,numRetentionOffersAccepted,newFrequentNumbers
0,26,yes,true,6,1,1812,false,0.00,C,true,...,0.33,0.00,0.362398,-6.330770,0.00,60,0.00,0,0,0
1,36,yes,true,9,4,544,true,79.99,A,false,...,4.00,1.67,5.586826,7.499560,0.00,53,0.00,0,0,0
2,74,yes,false,7,2,138,true,29.99,A,false,...,0.00,0.33,0.500000,-5.940493,4.33,58,0.00,0,0,3
3,30,no,false,6,3,122,true,9.99,C,false,...,9.00,4.67,8.629661,-0.066485,5.33,50,0.33,0,0,0
4,32,yes,false,7,8,10,true,129.99,A,false,...,59.33,23.00,14.042685,-7.208502,14.67,56,0.00,0,0,0


# Data looks more cleaner now

In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,10000.0,30.318400,22.158676,0.000000,0.000000,34.000000,48.000000,98.000000
income,10000.0,4.293600,3.139902,0.000000,0.000000,5.000000,7.000000,9.000000
numHandsets,10000.0,1.804500,1.345088,1.000000,1.000000,1.000000,2.000000,21.000000
handsetAge,10000.0,390.171700,257.076656,-5.000000,210.000000,339.000000,525.000000,1812.000000
currentHandsetPrice,10000.0,35.730696,57.072922,0.000000,0.000000,0.000000,59.990000,499.990000
avgBill,10000.0,58.927600,43.889815,0.000000,33.327500,49.205000,71.762500,584.230000
avgMins,10000.0,521.170645,540.435285,0.000000,150.627500,359.625000,709.187500,6336.250000
avgrecurringCharge,10000.0,46.236537,23.964960,0.000000,30.000000,44.990000,59.990000,337.980000
avgOverBundleMins,10000.0,42.392670,106.374374,0.000000,0.000000,3.000000,43.750000,4320.750000
avgRoamCalls,10000.0,1.186048,6.048811,0.000000,0.000000,0.000000,0.260000,177.990000


In [16]:
df_check = df.copy()

In [ ]:
df['marriageStatus'].value_counts()

In [ ]:
df['income'].value_counts()

In [ ]:
# hence income is being taken as categorical

In [ ]:
df['numHandsets'].value_counts()

In [ ]:
df["newFrequentNumbers"].value_counts()

In [ ]:
df["numRetentionOffersAccepted"].value_counts()

In [ ]:
df["numRetentionCalls"].value_counts()

In [ ]:
df["lastMonthCustomerCareCalls"].value_counts()

In [ ]:
# Based on the above, segregating Continious and Categorical

In [ ]:
list_cont = ["age", "handsetAge", "currentHandsetPrice","avgBill","avgMins","avgrecurringCharge","avgOverBundleMins","avgRoamCalls","callMinutesChangePct","billAmountChangePct","avgReceivedMins","avgOutCalls","avgInCalls","peakOffPeakRatio","peakOffPeakRatioChangePct","avgDroppedCalls","lifeTime","lastMonthCustomerCareCalls"]
list_cat =["income","numHandsets","marriageStatus","children","smartPhone","creditRating","homeOwner","creditCard","numRetentionCalls","numRetentionOffersAccepted","newFrequentNumbers"]

In [ ]:
#Treat the Continuous values, fill NA with the median value

for variable in list_cont:
    df[variable].fillna(df[variable].median(), inplace = True)

In [ ]:
# Apply label encoder for Categorical Variables.

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for i in list_cat:
    df[i] = labelencoder.fit_transform(df[i])

In [ ]:
df.head()

In [ ]:
# Create a Continuious Variable Dataset, to view KDE and later apply z-score

df_cont = df.drop(["income","numHandsets","marriageStatus","children","smartPhone","creditRating","homeOwner","creditCard","numRetentionCalls","numRetentionOffersAccepted","newFrequentNumbers"],axis=1)

In [ ]:
df_cont.shape

In [ ]:
from matplotlib.colors import ListedColormap
import seaborn as sns; sns.set()
plt.figure(figsize=(20,20))
# my_cmap = sns.light_palette("Violet", as_cmap=True)
# my_cmap = ListedColormap(sns.husl_palette(7))
my_cmap = ListedColormap(sns.mpl_palette("Blues"))
ax = sns.heatmap(df.corr(), vmax=.8, fmt='.2f', annot=True, square=True, cmap=my_cmap)
plt.title('Correlation Value')
plt.show()

# Drop high multicollinearity columns

From the above we can see the following have high co-relation

Income-CreditCard

AvgMins-AvgBill-AvgOverBundleMins

AvgReceiveMins - AvgMins

NumberOfRetentionOffersAccepted - NumberOfRetentionCalls



In [ ]:
df.drop(["income","avgBill","avgReceivedMins","numRetentionOffersAccepted"],axis=1,inplace=True)

In [ ]:
df2 = df.copy()
df2 = df.drop(["numHandsets","marriageStatus","children","smartPhone","creditRating","homeOwner","creditCard","numRetentionCalls","newFrequentNumbers"],axis=1)

In [ ]:
import seaborn as sns
sns.pairplot(df2,diag_kind='kde')

In [ ]:
#Apply zscore 
df_scaled = df2.apply(zscore)

In [17]:
model = KMeans(n_clusters = 10)

In [18]:
cluster_range = range( 1, 10 )
cluster_errors = []
for num_clusters in cluster_range:
  clusters = KMeans( num_clusters, n_init = 10 )
  clusters.fit(df_check)
  labels = clusters.labels_
  centroids = clusters.cluster_centers_
  cluster_errors.append( clusters.inertia_ )
clusters_df = pd.DataFrame( { "num_clusters":cluster_range, "cluster_errors": cluster_errors } )
clusters_df[0:10]

ValueError: could not convert string to float: 'true'

In [ ]:
# Elbow plot

plt.figure(figsize=(20,10))
plt.plot( clusters_df.num_clusters, clusters_df.cluster_errors, marker = "o")

In [ ]:
# From the above we take n_clusters as 3

In [ ]:
kmeans = KMeans(n_clusters=3, n_init = 15, random_state=42)

In [ ]:
kmeans.fit(df_scaled)

In [ ]:
centroids = kmeans.cluster_centers_

In [ ]:
centroid_df = pd.DataFrame(centroids, columns = list(df_scaled) )

In [ ]:
centroid_df

In [ ]:
## creating a new dataframe only for labels and converting it into categorical variable
df_labels = pd.DataFrame(kmeans.labels_ , columns = list(['labels']))

df_labels['labels'] = df_labels['labels'].astype('category')

In [ ]:
# Joining the label dataframe with the Wine data frame to create wine_df_labeled. Note: it could be appended to original dataframe
churn_df_labeled = df2.join(df_labels)

In [ ]:
df_analysis = (churn_df_labeled.groupby(['labels'] , axis=0)).head(1599)  # the groupby creates a groupeddataframe that needs 
# to be converted back to dataframe. I am using .head(30000) for that
df_analysis

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = Axes3D(fig, rect=[0, 0, 1, 1], elev=-45, azim=100)
kmeans.fit(df_scaled)
labels = kmeans.labels_

ax.scatter(df_scaled.iloc[:, 3], df_scaled.iloc[:, 9], df_scaled.iloc[:, 10],c=labels.astype(np.float), edgecolor='k')
ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_xlabel('avgMins')
ax.set_ylabel('avgInCalls')
ax.set_zlabel('avgOutCalls')
ax.set_title('3D plot of KMeans Clustering')

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = Axes3D(fig, rect=[.80, .75, .90, 1], elev=-60, azim=120)
kmeans.fit(df_scaled)
labels = kmeans.labels_

ax.scatter(df_scaled.iloc[:, 1], df_scaled.iloc[:, 9], df_scaled.iloc[:, 13],c=labels.astype(np.float), edgecolor='k')
ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_xlabel('handsetAge')
ax.set_ylabel('avgInCalls')
ax.set_zlabel('avgDropCalls')
ax.set_title('3D plot of KMeans Clustering')

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = Axes3D(fig, rect=[.80, .75, .90, 1], elev=60, azim=120)
kmeans.fit(df_scaled)
labels = kmeans.labels_

ax.scatter(df_scaled.iloc[:, 5], df_scaled.iloc[:, 9], df_scaled.iloc[:, 1],c=labels.astype(np.float), edgecolor='k')
ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_xlabel('avgOverBundleMins')
ax.set_ylabel('avgInCalls')
ax.set_zlabel('handsetAge')

ax.set_title('3D plot of KMeans Clustering')

In [ ]:
import matplotlib.pylab as plt

bxplt = churn_df_labeled.boxplot(by = 'labels',  layout=(4,6), figsize=(25, 25))
bxplt

# We can see that Avgerage Minutes, Handset Age, and Average Over Bundle Minutes are relatively good indicators to cluster the customers for clustering the customers.

# Also, looking at the dataset, Number of Retention Calls also has a direct relation with Churn